In [2]:
import pandas as pd
from datetime import datetime
import win32com.client as win32
import os

# Proporções de cada grupo
proporcoes_grupo = {
    "Backoffice": 0.60,
    "Intimações": 0.25,
    "Consolidações": 0.15
}

# Proporções dentro de cada grupo
proporcoes_advogados = {
    "Backoffice": {
        "Ana Carolina Bressan da Silva": 0.30,
        "Erick Damin Bitencourt": 0.40,
        "Rayana Eduarda Da Silva Ferreira" : 0.30
        
    },
    "Intimações": {
        "Maíra Vogt Silva": 0.25,
        "João Lucas Martins Falcão": 0.25,
        "Darlei Jacoby Kayser": 0.25,
        "Victor Lopes Machado Gonçalves": 0.25
    },
    "Consolidações": {
        "Rafaella Rodrigues dos Santos Marques": 0.20,
        "Bruna Ariane Pereira Martins": 0.20,
        "Danielle Lais da Silva Lutkemeyer": 0.20,
        "Matheus Cezar Dias": 0.20,
        "Tatiane de Souza" : 0.20
    }
}

# Mapeamento de advogados e e-mails
email_por_advogado = {
    "Ana Carolina Bressan da Silva": "pedrocecere@ramaadvogados.com.br",
    "Darlei Jacoby Kayser": "pedrocecer@ramaadvogados.com.br",
    "Erick Damin Bitencourt": "pedrocecer@ramaadvogados.com.br",
    "Danielle Lais da Silva Lutkemeyer": "pedrocecer@ramaadvogados.com.br",
    "Rafaella Rodrigues dos Santos Marques": "pedrocecer@ramaadvogados.com.br",
    "Matheus Cezar Dias": "pedrocecer@ramaadvogados.com.br",
    "João Lucas Martins Falcão" : "pedrocecer@ramaadvogados.com.br",
    "Victor Lopes Machado Gonçalves" : "pedrocecer@ramaadvogados.com.br",
    "Bruna Ariane Pereira Martins" : "pedrocecer@ramaadvogados.com.br",
    "Maíra Vogt Silva" : "pedrocecer@ramaadvogados.com.br",
    "Tatiane de Souza" : "pedrocecer@ramaadvogados.com.br",
    "Bruna Ariane Pereira Martins" : "pedrocecer@ramaadvogados.com.br",
    "Rayana Eduarda Da Silva Ferreira" : "pedrocecer@ramaadvogados.com.br"
}

# Inicializa o Outlook
outlook = win32.Dispatch('Outlook.Application')
remetente = 'pedro.cecere@ramaadvogados.com.br'
assunto = 'Relatório por Advogado'

def tratamento_dados(dados):
    # Filtrando registros que não são de interesse
    if 'Subase' in dados.columns:
        dados_filtrados = dados[dados['Subase'] != "AF 12 - Pagamento"]
    else:
        print("A coluna 'Subase' não foi encontrada.")
        return None

    # Removendo registros do mês atual
    if 'Data atualização Benner' in dados_filtrados.columns:
        dados_filtrados['Data atualização Benner'] = pd.to_datetime(dados_filtrados['Data atualização Benner'])
        mes_atual = datetime.now().month    
        dados_filtrados = dados_filtrados.loc[dados_filtrados['Data atualização Benner'].dt.month != mes_atual]
    else:
        print("A coluna 'Data atualização Benner' não foi encontrada.")
        return None
    
    # Mapeando núcleo por advogado
    nucleo_por_advogado = {
        "Ana Carolina Bressan da Silva": "Backoffice",
        "Darlei Jacoby Kayser": "Intimações",
        "Erick Damin Bitencourt": "Backoffice",
        "Rafaella Rodrigues dos Santos Marques": "Consolidações",
        "Danielle Lais da Silva Lutkemeyer": "Consolidações",
        "Matheus Cezar Dias": "Backoffice",
        "João Lucas Martins Falcão" : "Intimação",
        "Victor Lopes Machado Gonçalves" : "Backoffice",
        "Maíra Vogt Silva" : "Intimação",
        "Rayana Eduarda Da Silva Ferreira" : "Backoffice",
        "Bruna Ariane Pereira Martins" : "Consolidações",
        "Tatiane de Souza" : "Consolidações"
        
    }
    
    dados_filtrados['Núcleo'] = dados_filtrados['Advogado interno'].map(nucleo_por_advogado)

    return dados_filtrados

def dividir_base_por_grupos(df, proporcoes_grupo):
    base_dividida = {}
    total_registros = len(df)
    print(total_registros)

    for grupo, proporcao in proporcoes_grupo.items():
        tamanho_grupo = int(total_registros * proporcao)
        base_dividida[grupo] = df.sample(n=tamanho_grupo, random_state=42)
        df = df.drop(base_dividida[grupo].index)

    return base_dividida

def dividir_base_por_advogados(base_dividida, proporcoes_advogados):
    base_final = {}

    for grupo, df_grupo in base_dividida.items():
        total_registros_grupo = len(df_grupo)
        base_final[grupo] = {}

        for advogado, proporcao in proporcoes_advogados[grupo].items():
            tamanho_advogado = int(total_registros_grupo * proporcao)
            base_final[grupo][advogado] = df_grupo.sample(n=tamanho_advogado, random_state=42)
            df_grupo = df_grupo.drop(base_final[grupo][advogado].index)

    return base_final

def salvar_e_enviar_arquivos(base_final, email_por_advogado):
    for grupo, advogados in base_final.items():
        for advogado, df_advogado in advogados.items():
            # Salvar o arquivo
            file_name = f"{advogado}_{grupo}.xlsx"
            df_advogado.to_excel(file_name, index=False)
            
            # Enviar o e-mail com o arquivo anexo
            enviar_email_com_anexo(email_por_advogado[advogado], file_name)

def enviar_email_com_anexo(email_destino, file_name):
    mail = outlook.CreateItem(0)
    mail.Subject = assunto
    mail.Body = "Olá! Segue anexo sua base de atualização WF"
    mail.To = email_destino
    mail.Attachments.Add(os.path.join(os.getcwd(), file_name))
    mail.Send()

if __name__ == "__main__":
    # Carregue a base de dados
    df = pd.read_excel(r"C:\Users\pedro.cecere\Downloads\data (9).xlsx", header=0)
    df = tratamento_dados(df)
    
    # Divida a base por grupos e depois por advogados
    base_dividida = dividir_base_por_grupos(df, proporcoes_grupo)
    base_final = dividir_base_por_advogados(base_dividida, proporcoes_advogados)
    
    # Salve os arquivos e envie por e-mail
    salvar_e_enviar_arquivos(base_final, email_por_advogado)


405
